<a href="https://colab.research.google.com/github/rashi311/capstone_project/blob/main/capstone_project_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install pathway bokeh --quiet


In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime,timedelta
import pathway as pw
import bokeh.plotting
import panel as pn

In [72]:
df=pd.read_csv("dataset.csv")

In [73]:
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [74]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [75]:
# Save required columns for streaming
# MODEL 1 requires only Occupancy and Capacity
# MODEL 2 uses additional demand-related columns

# For Model 1 and Model 1B
model1_df = df[["Timestamp", "Occupancy", "Capacity"]]
model1_df.to_csv("parking_stream_model1.csv", index=False)

In [76]:

# For Model 2 - Create some synthetic demand-related fields
np.random.seed(42)
df['QueueLength'] = np.random.randint(0, 20, size=len(df))
df['Traffic'] = np.random.randint(1, 10, size=len(df))
df['IsSpecialDay'] = np.random.choice([0, 1], size=len(df), p=[0.9, 0.1])
df['VehicleTypeWeight'] = np.random.uniform(0.8, 1.2, size=len(df))
model2_df = df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "Traffic", "IsSpecialDay", "VehicleTypeWeight"]]
model2_df.to_csv("parking_stream_model2.csv", index=False)

In [77]:
# MODEL 1: BASELINE LINEAR PRICING
class ParkingSchema1(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

data1 = pw.demo.replay_csv("parking_stream_model1.csv", schema=ParkingSchema1, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data1_time = data1.with_columns(
    t = data1.Timestamp.dt.strptime(fmt),
    day = data1.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

base_price = 10
alpha = 5

linear_price = data1_time.with_columns(
    occ_ratio = data1_time.Occupancy / data1_time.Capacity,
    price = base_price + alpha * (data1_time.Occupancy / data1_time.Capacity)
)


In [78]:
def plot_model1(source):
    fig = bokeh.plotting.figure(height=400, width=800, title="Model 1: Linear Pricing", x_axis_type="datetime")
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.scatter("t", "price", source=source, size=5, color="red")
    return fig

viz1 = linear_price.plot(plot_model1, sorting_col="t")

In [79]:
# MODEL 1B: DAILY FLUCTUATION PRICING
delta_window = (
    data1_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)

In [80]:
def plot_model1b(source):
    fig = bokeh.plotting.figure(height=400, width=800, title="Model 1B: Daily Fluctuation Pricing", x_axis_type="datetime")
    fig.line("t", "price", source=source, line_width=2, color="purple")
    fig.scatter("t", "price", source=source, size=5, color="black")
    return fig

viz1b = delta_window.plot(plot_model1b, sorting_col="t")

In [83]:
# MODEL 2: DEMAND-BASED PRICING

class ParkingSchema2(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: int
    IsSpecialDay: int
    VehicleTypeWeight: float

data2 = pw.demo.replay_csv("parking_stream_model2.csv", schema=ParkingSchema2, input_rate=1000)

data2_time = data2.with_columns(
    t = data2.Timestamp.dt.strptime(fmt),
    day = data2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

windowed = (
    data2_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_avg=pw.reducers.avg(pw.this.Occupancy),
        queue_avg=pw.reducers.avg(pw.this.QueueLength),
        traffic_avg=pw.reducers.avg(pw.this.Traffic),
        vt_weight_avg=pw.reducers.avg(pw.this.VehicleTypeWeight),

        special_day=pw.reducers.max(pw.this.IsSpecialDay),

        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price = 10 \
            + (pw.this.occ_avg / pw.this.cap) * 5 \
            + pw.this.queue_avg * 0.3 \
            + pw.this.traffic_avg * 0.2 \
            + pw.if_else(pw.this.special_day == 1, 5, 0) \
            + (pw.this.vt_weight_avg - 1) * 10
    )
)

In [85]:
def plot_model2(source):
    fig = bokeh.plotting.figure(height=400, width=800, title="Model 2: Demand-Based Pricing", x_axis_type="datetime")
    fig.line("t", "price", source=source, line_width=2, color="blue")
    fig.scatter("t", "price", source=source, size=5, color="orange")
    return fig

viz2 = windowed.plot(plot_model2, sorting_col="t")

In [87]:
pn.extension()
# Create dashboard layout



pn.Column("# Model 1 & 2 Pricing Visualization", viz1, viz2).servable()

Column
    [0] Markdown(str)
    [1] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)
    [2] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [88]:
%%capture --no-display
pw.run()

Output()